# [Tool calling](https://python.langchain.com/docs/concepts/tool_calling/)

###### Prerequisites
- [Tools]()
- [Chat Models]()


## Overview
- Many AI applications interact directly with humans. In these cases, it is appropriate for models to respond in natural language. But what about cases where we want a model to also interact directly with systems, such as databases or an API? These systems often have a particular input schema; for example, APIs frequently have a required payload structure. This need motivates the concept of tool calling. You can use tool calling to request model responses that match a particular schema.

![](./images/01.png)

In [1]:
%load_ext autoreload
%autoreload 2

## Key concepts
- **Tool Creation**:  Use the `@tool` decorator to create a tool. A tool is an association between **a function** and **its schema**.
- **Tool Binding**: The tool needs to be connected to a model that supports tool calling. This gives the model awareness of the tool and the associated input schema required by the tool.
- **Tool Calling**: When appropriate, the model can decide to call a tool and ensure its response conforms to the tool's input schema.
- **Tool Execution**: The tool can be executed using the arguments provided by the model.

![](./images/02.png)

## Recommended usage
- This pseudo-code illustrates the recommended workflow for using tool calling. Created tools are passed to .bind_tools() method as a list. This model can be called, as usual. If a tool call is made, model's response will contain the tool call arguments. The tool call arguments can be passed directly to the tool.
  ```python
  # Tool creation
  tools = [my_tool]
  # Tool binding
  model_with_tools = model.bind_tools(tools)
  # Tool calling 
  response = model_with_tools.invoke(user_input)
  ```

## Tool creation
- The recommended way to create a tool is using the `@tool` decorator.

In [2]:
from langchain_core.tools import tool

In [3]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    Returns:
        int: the product of a and b
    """
    return a * b

# Tool binding
- The central concept to understand is that LangChain provides a standardized interface for connecting tools to models. The `.bind_tools()` method can be used to specify which tools are available for a model to call.
- As a specific example, let's take a function `multiply` and bind it as a tool to a model that supports tool calling.

In [4]:
def load_env_to_dict(file_path):
    env_dict = {}
    with open(file_path, "r") as file:
        for line in file:
            # Remove whitespace and ignore comments or empty lines
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            # Split the line into key and value
            key, value = line.split("=", 1)
            env_dict[key.strip()] = value.strip()
    return env_dict

In [5]:
file_path = "/mnt/Exdisk/git-cuongpiger/secret/work/vngcloud/ai-platform/env"
env_variables = load_env_to_dict(file_path)

In [6]:
import os

from langchain_google_vertexai import ChatVertexAI
from langchain_ollama import ChatOllama

In [7]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "/mnt/Exdisk/git-cuongpiger/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
)

In [8]:
llm = ChatVertexAI(model="gemini-1.5-pro")

In [9]:
tools_list = [multiply]

In [10]:
llm_bind_tools = llm.bind_tools(tools_list)

# Tool calling
- A key principle of tool calling is that the model decides when to use a tool based on the input's relevance. The model doesn't always need to call a tool. For example, given an unrelated input, the model would not call the tool:

  ![](./images/03.png)

In [14]:
result = llm_bind_tools.invoke("Hello world!")
result

AIMessage(content='Hello world! What can I do for you today? 😊 \n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.06103515625, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.04150390625}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.083984375, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.06298828125}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.10986328125, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.053466796875}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.25, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.0849609375}], 'usage_metadata': {'prompt_tok

The result would be an `AIMessage` containing the model's response in natural language (e.g., "Hello!"). However, if we pass an input relevant to the tool, the model should choose to call it:

In [15]:
result = llm_bind_tools.invoke("What is 2 multiplied by 3?")
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 3.0}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.1240234375, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.09521484375}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.1357421875, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.1025390625}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.17578125, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.1142578125}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.0673828125, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.07470703125}],

As before, the output result will be an `AIMessage`. But, if the tool was called, `result` will have a `tool_calls` attribute. This attribute includes everything needed to execute the tool, including the tool name and input arguments:



In [16]:
result.tool_calls

[{'name': 'multiply',
  'args': {'a': 2.0, 'b': 3.0},
  'id': '5ad85f03-83de-4f3c-bf6f-97b60cae614c',
  'type': 'tool_call'}]

## Tool execution
- Tools implement the `Runnable` interface, which means that they can be invoked (e.g., `tool.invoke(args)`) directly.
- LangGraph offers pre-built components (e.g., `ToolNode`) that will often invoke the tool in behalf of the user.

## Best practices
- When designing tools to be used by a model, it is important to keep in mind that:
  - Models that have explicit **tool-calling APIs** will be better at tool calling than non-fine-tuned models.
  - Models will perform better if the tools have well-chosen names and descriptions.
  - Simple, narrowly scoped tools are easier for models to use than complex tools.
  - Asking the model to select from a large list of tools poses challenges for the model.